In [2]:
from keras.models import *
from keras.layers import *
from keras.optimizers import RMSprop,SGD,Adadelta,Adam
from keras.preprocessing import image
from keras.callbacks import TensorBoard,History,EarlyStopping,CSVLogger
from keras.applications.vgg16 import VGG16,preprocess_input
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import h5py
import os

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [3]:
train_path = './train'
validation_path = './validation'
test_path = './test'
assert(os.path.exists(train_path))
assert(os.path.exists(validation_path))
assert(os.path.exists(test_path))


nb_train_samples = 20000
nb_val_samples = 2424
image_size = (224,224)
batch_size = 128
class_num = 10

In [4]:
vgg16_base_model = VGG16(weights='imagenet', include_top=False,input_shape=(224,224,3))

x = vgg16_base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
# x = Dense(256, activation='relu')(x)
# x = Dropout(0.5)(x)
# x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax')(x)

vgg16_model = Model(inputs=vgg16_base_model.input, outputs=x)

In [5]:
for layer in vgg16_model.layers[:19]:
    layer.trainable = False

for i,layer in enumerate(vgg16_model.layers):
    print(i,layer.name,layer.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 False
16 block5_conv2 False
17 block5_conv3 False
18 block5_pool False
19 flatten_1 True
20 dense_1 True
21 batch_normalization_1 True
22 dropout_1 True
23 dense_2 True
24 batch_normalization_2 True
25 dropout_2 True
26 dense_3 True


In [6]:
train_datagen = image.ImageDataGenerator(
                                         shear_range=0.15,
                                         rotation_range=15,
                                         height_shift_range=0.15,
                                         width_shift_range=0.15,
                                         channel_shift_range=10,
                                         preprocessing_function=preprocess_input,
                                         horizontal_flip=True)

validation_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = train_datagen.flow_from_directory(
                                    train_path,
                                    target_size=image_size,
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
                                    validation_path,
                                    target_size=image_size,
                                    batch_size=batch_size,                                    
                                    class_mode='categorical',
                                    shuffle=False)

Found 20000 images belonging to 10 classes.
Found 2424 images belonging to 10 classes.


In [7]:
vgg16_model.compile(loss='categorical_crossentropy',
                    #optimizer=optimizers.SGD(lr=1e-3, momentum=0.9) #,decay=1e-6,nesterov=True),
                    #optimizer=optimizers.Adam(lr=1e-3),
                    optimizer=Adadelta(),
                    metrics=['accuracy'])

# fine-tune the model
vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=10,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[EarlyStopping(patience=3)])

Epoch 1/10
156/156 [==============================] - 505s 3s/step - loss: 1.4738 - acc: 0.5329 - val_loss: 1.0934 - val_acc: 0.6871
Epoch 2/10
156/156 [==============================] - 352s 2s/step - loss: 0.5718 - acc: 0.8093 - val_loss: 1.0747 - val_acc: 0.6925
Epoch 3/10
156/156 [==============================] - 340s 2s/step - loss: 0.3986 - acc: 0.8677 - val_loss: 1.0878 - val_acc: 0.7073
Epoch 4/10
156/156 [==============================] - 341s 2s/step - loss: 0.3076 - acc: 0.9002 - val_loss: 1.4783 - val_acc: 0.6638
Epoch 5/10
156/156 [==============================] - 340s 2s/step - loss: 0.2698 - acc: 0.9123 - val_loss: 1.5346 - val_acc: 0.6220


In [9]:
# vgg16_model.optimizer.lr = 1e-4

# vgg16_model.fit_generator(
#                         train_generator,
#                         steps_per_epoch=nb_train_samples // batch_size,
#                         epochs=5,
#                         validation_data=validation_generator,
#                         validation_steps=nb_val_samples // batch_size)

Epoch 1/5
156/156 [==============================] - 344s 2s/step - loss: 0.1321 - acc: 0.9568 - val_loss: 1.1900 - val_acc: 0.7036
Epoch 2/5
156/156 [==============================] - 344s 2s/step - loss: 0.1315 - acc: 0.9580 - val_loss: 1.5762 - val_acc: 0.6494
Epoch 3/5
156/156 [==============================] - 345s 2s/step - loss: 0.1116 - acc: 0.9632 - val_loss: 1.6493 - val_acc: 0.6071
Epoch 4/5
156/156 [==============================] - 341s 2s/step - loss: 0.1080 - acc: 0.9657 - val_loss: 1.7442 - val_acc: 0.6150
Epoch 5/5
156/156 [==============================] - 341s 2s/step - loss: 0.1055 - acc: 0.9660 - val_loss: 2.0907 - val_acc: 0.5675


In [8]:
for layer in vgg16_model.layers[15:]:
    layer.trainable = True
    
vgg16_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=0.0001, momentum=0.9),#,decay=1e-6,nesterov=True),
                    optimizer=optimizers.Adam(lr=1e-3),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                   EarlyStopping(patience=5)])


vgg16_model.save('vgg16_fine_tuning_model.h5')

Epoch 1/20
156/156 [==============================] - 347s 2s/step - loss: 0.7079 - acc: 0.8005 - val_loss: 14.5707 - val_acc: 0.0625
Epoch 2/20
156/156 [==============================] - 344s 2s/step - loss: 0.4079 - acc: 0.8672 - val_loss: 6.6555 - val_acc: 0.2178
Epoch 3/20
156/156 [==============================] - 346s 2s/step - loss: 0.1506 - acc: 0.9558 - val_loss: 2.6541 - val_acc: 0.5658
Epoch 4/20
156/156 [==============================] - 344s 2s/step - loss: 0.0983 - acc: 0.9723 - val_loss: 2.0548 - val_acc: 0.5248
Epoch 5/20
156/156 [==============================] - 343s 2s/step - loss: 0.0667 - acc: 0.9804 - val_loss: 1.5825 - val_acc: 0.6899
Epoch 6/20
156/156 [==============================] - 343s 2s/step - loss: 0.0567 - acc: 0.9840 - val_loss: 1.6855 - val_acc: 0.6760
Epoch 7/20
156/156 [==============================] - 341s 2s/step - loss: 0.1021 - acc: 0.9692 - val_loss: 1.9993 - val_acc: 0.6115
Epoch 8/20
156/156 [==============================] - 343s 2s/step -

In [9]:
vgg16_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.SGD(lr=1e-4, momentum=0.9,decay=1e-6,nesterov=True),
#                     optimizer=optimizers.Adam(lr=1e-3),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                   EarlyStopping(patience=5)])

vgg16_model.save('vgg16_fine_tuning_model_SDG.h5')

Epoch 1/20
156/156 [==============================] - 346s 2s/step - loss: 0.0322 - acc: 0.9905 - val_loss: 1.3825 - val_acc: 0.7070
Epoch 2/20
156/156 [==============================] - 344s 2s/step - loss: 0.0351 - acc: 0.9897 - val_loss: 1.1948 - val_acc: 0.7265
Epoch 3/20
156/156 [==============================] - 344s 2s/step - loss: 0.0352 - acc: 0.9901 - val_loss: 1.1621 - val_acc: 0.7274
Epoch 4/20
156/156 [==============================] - 346s 2s/step - loss: 0.0342 - acc: 0.9891 - val_loss: 1.0850 - val_acc: 0.7374
Epoch 5/20
156/156 [==============================] - 343s 2s/step - loss: 0.0338 - acc: 0.9902 - val_loss: 1.1631 - val_acc: 0.7252
Epoch 6/20
156/156 [==============================] - 342s 2s/step - loss: 0.0332 - acc: 0.9906 - val_loss: 1.1610 - val_acc: 0.7239
Epoch 7/20
156/156 [==============================] - 341s 2s/step - loss: 0.0341 - acc: 0.9902 - val_loss: 1.2107 - val_acc: 0.7125
Epoch 8/20
156/156 [==============================] - 340s 2s/step - 

In [10]:
for layer in vgg16_model.layers[11:15]:
    layer.trainable = True
    
for i,layer in enumerate(vgg16_model.layers):
    print(i,layer.name,layer.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 True
12 block4_conv2 True
13 block4_conv3 True
14 block4_pool True
15 block5_conv1 True
16 block5_conv2 True
17 block5_conv3 True
18 block5_pool True
19 flatten_1 True
20 dense_1 True
21 batch_normalization_1 True
22 dropout_1 True
23 dense_2 True
24 batch_normalization_2 True
25 dropout_2 True
26 dense_3 True


In [11]:
vgg16_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.SGD(lr=1e-4, momentum=0.9,decay=1e-6,nesterov=True),
#                     optimizer=optimizers.Adam(lr=1e-3),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                   EarlyStopping(patience=5)])

vgg16_model.save('vgg16_fine_tuning_model_11_15.h5')

Epoch 1/20
156/156 [==============================] - 352s 2s/step - loss: 0.0288 - acc: 0.9918 - val_loss: 1.5114 - val_acc: 0.6580
Epoch 2/20
156/156 [==============================] - 349s 2s/step - loss: 0.0262 - acc: 0.9933 - val_loss: 1.4071 - val_acc: 0.6877
Epoch 3/20
156/156 [==============================] - 348s 2s/step - loss: 0.0258 - acc: 0.9933 - val_loss: 1.2418 - val_acc: 0.7139
Epoch 4/20
156/156 [==============================] - 351s 2s/step - loss: 0.0209 - acc: 0.9948 - val_loss: 1.4969 - val_acc: 0.6764
Epoch 5/20
156/156 [==============================] - 348s 2s/step - loss: 0.0234 - acc: 0.9944 - val_loss: 1.3445 - val_acc: 0.6973
Epoch 6/20
 15/156 [=>............................] - ETA: 4:46 - loss: 0.0176 - acc: 0.9953

KeyboardInterrupt: 